In [1]:
# Dependencies
import pandas as pd
import requests
from pprint import pprint
import pymongo
from bs4 import BeautifulSoup as bs
from splinter import Browser
from flask import Flask, render_template
import lxml

# Nasa's Mars News Site
url = "https://mars.nasa.gov/news/"



In [2]:
# Set up MongoDB
conn='mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Retrieve page with the requests module
response = requests.get(url)
html = response.text
    
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [4]:
# Assign variables
news_title = soup.find(class_='content_title')
news_p = soup.find(class_='article_teaser_body')

In [5]:
# Splinter to featured image
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(featured_image_url)

In [6]:
# Twitter
twitter_url = 'https://twitter.com/marswxreport?lang=en'
response1 = requests.get(twitter_url)
html = response1.text

soup1 = bs(html, 'html.parser')
tweet = soup1.find(class_='css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0')

In [7]:
# Mars Facts table
facts_url = 'https://space-facts.com/mars/'
mars_table = pd.read_html(facts_url)
mars_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [8]:
#Hemisphere images
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere",\
     "img_url": "https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced"},
    {"title": "Cerberus Hemisphere",\
     "img_url": "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced"},
    {"title": "Schiaparelli Hemisphere",\
     "img_url": "https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced"},
    {"title": "Syrtis Major Hemisphere",\
     "img_url": "https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced"},
]


In [9]:
browser.quit

<bound method BaseWebDriver.quit of <splinter.driver.webdriver.chrome.WebDriver object at 0x0000027AE38BC188>>